In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`

import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.rdd.RDD
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .master("local[*]").appName("Validator")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                    

import $ivy.$                                

import $ivy.$                              


import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.rdd.RDD

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@5a61213e
import spark.implicits._

appID: String = "local-1556122525216"

In [13]:
val path = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
val state = "NY"

path: String = "/home/acald013/RIDIR/Datasets/AreaTablesValidation"
state: String = "NY"

In [14]:
val geopandas = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geopandas_test.tsv").distinct()
geopandas.count()

geopandas: Dataset[Row] = [_c0: string, _c1: string ... 1 more field]
res13_1: Long = 8151L

In [15]:
val geospark = spark.read.option("header", "false").option("delimiter", "\t").csv(s"${path}/${state}_geospark_test.tsv").distinct()
geospark.count()

geospark: Dataset[Row] = [_c0: string, _c1: string ... 1 more field]
res14_1: Long = 8151L

In [16]:
val p = geopandas.toDF("s1", "t1", "area1").orderBy("s1", "t1", "area1")
val s = geopandas.toDF("s2", "t2", "area2").orderBy("s2", "t2", "area2")
val areas = p.select("area1").map(_.getString(0).toDouble).rdd.zip(s.select("area2").map(_.getString(0).toDouble).rdd)

p: Dataset[Row] = [s1: string, t1: string ... 1 more field]
s: Dataset[Row] = [s2: string, t2: string ... 1 more field]
areas: RDD[(Double, Double)] = ZippedPartitionsRDD2[191] at zip at cmd15.sc:3

In [17]:
areas.toDF("area1", "area2").show(truncate = false)

+---------------------+---------------------+
|area1                |area2                |
+---------------------+---------------------+
|2.5261844522811287E-4|2.5261844522811287E-4|
|1.5625384114862862E-9|1.5625384114862862E-9|
|2.739295819020814E-6 |2.739295819020814E-6 |
|3.460224149717579E-6 |3.460224149717579E-6 |
|5.562846016544312E-6 |5.562846016544312E-6 |
|6.0613064292227E-7   |6.0613064292227E-7   |
|7.053096052975711E-7 |7.053096052975711E-7 |
|1.5807291728437404E-4|1.5807291728437404E-4|
|3.136674887704342E-6 |3.136674887704342E-6 |
|1.4531710548415795E-6|1.4531710548415795E-6|
|1.5161355047734145E-7|1.5161355047734145E-7|
|0.011766889918222573 |0.011766889918222573 |
|5.807427196973919E-5 |5.807427196973919E-5 |
|0.017480647972263185 |0.017480647972263185 |
|8.105140766924522E-6 |8.105140766924522E-6 |
|4.167211632987942E-4 |4.167211632987942E-4 |
|2.1286769616081068E-6|2.1286769616081068E-6|
|8.20496423948473E-6  |8.20496423948473E-6  |
|4.828190247372042E-4 |4.828190247

In [18]:
val reg = new RegressionMetrics(areas)
reg.r2
reg.meanAbsoluteError
reg.meanSquaredError
reg.rootMeanSquaredError

reg: RegressionMetrics = org.apache.spark.mllib.evaluation.RegressionMetrics@34c93c6
res17_1: Double = 1.0
res17_2: Double = 0.0
res17_3: Double = 0.0
res17_4: Double = 0.0